In [ ]:
import os
import cv2
import json
import numpy as np
import subprocess
from datetime import datetime

# Path utama untuk OpenPose dan direktori gambar
openpose_path = os.path.abspath("openpose")  # Path ke OpenPose
image_dir = os.path.abspath("../images")  # Folder gambar input
json_output_dir = os.path.abspath("../output_json")  # Folder JSON hasil OpenPose
preprocessing_dir = os.path.abspath("../preprocessing_images")  # Folder hasil crop
log_dir = os.path.abspath("../log_preprocessing")

# Tambahkan timestamp pada nama file log
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
log_file_path = os.path.join(log_dir, f"error_log_preprocessing_{timestamp}.txt")

# Buat folder jika belum ada
os.makedirs(image_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(preprocessing_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)

# Ambil file gambar dari folder `images/`
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]

if not image_files:
    print("Tidak ada gambar yang valid di folder `images/`. Silakan unggah file JPG atau PNG!")
    exit()

# hitung jumlah gambar
total_images = len(image_files)
print(f"Preprocessing of {total_images} images!, please wait...")

# Jalankan OpenPose dengan `cwd=openpose_path` (tanpa menyimpan gambar hasil OpenPose)
cmd = [
    "bin\\OpenPoseDemo.exe",
    "--image_dir", image_dir,
    "--write_json", json_output_dir,
    "--display", "0",
    "--render_pose", "0",
    "--model_pose", "BODY_25",
    "--number_people_max", "1"
]

result = subprocess.run(cmd, shell=True, capture_output=True, text=True, cwd=openpose_path)

# **Periksa apakah OpenPose berjalan dengan sukses**
if result.returncode != 0:
    error_msg = "OpenPose gagal dijalankan! Periksa konfigurasi atau file gambar."
    print(error_msg)
    log_file.write(error_msg)
    exit()

# **Periksa apakah ada JSON hasil OpenPose**
json_files = [f for f in os.listdir(json_output_dir) if f.endswith(".json")]
if not json_files:
    error_msg = "Tidak ada JSON yang dihasilkan oleh OpenPose! Pastikan gambar terdeteksi dengan benar."
    print(error_msg)
    log_file.write(error_msg)
    exit()

# Buka file log untuk mencatat error
with open(log_file_path, "w") as log_file:
    for index, image_file in enumerate(image_files, start=1):
        input_image_path = os.path.join(image_dir, image_file)
        file_name, file_ext = os.path.splitext(image_file)

        output_json_path = os.path.join(json_output_dir, file_name + '_keypoints.json')

        # Tampilkan progress
        print(f"Memproses ({index}/{total_images}): {image_file} ...")

        if result.returncode != 0:
            error_msg = f"Gagal memproses {image_file} dengan OpenPose!\n"
            print(error_msg)
            log_file.write(error_msg)
            continue

        # Baca hasil JSON dari OpenPose
        if not os.path.exists(output_json_path):
            error_msg = f"JSON hasil deteksi tidak ditemukan untuk {image_file}!\n"
            print(error_msg)
            log_file.write(error_msg)
            continue

        with open(output_json_path, 'r') as f:
            pose_data = json.load(f)
            if not pose_data['people']:
                error_msg = f"Tidak ada orang terdeteksi pada {image_file}.\n"
                print(error_msg)
                log_file.write(error_msg)
                continue
            keypoints = pose_data['people'][0]['pose_keypoints_2d']

        # Ambil koordinat x, y untuk semua keypoints
        def get_coords(index):
            x = keypoints[index * 3]
            y = keypoints[index * 3 + 1]
            confidence = keypoints[index * 3 + 2]
            return (int(x), int(y)) if confidence > 0 else None

        def get_coords_flip(index):
            return (keypoints[index * 3], keypoints[index * 3 + 1]) if keypoints and keypoints[index * 3 + 2] > 0 else None

        body_points = [get_coords(i) for i in range(25)]
        valid_points = [p for p in body_points if p is not None]

        if not valid_points:
            error_msg = f"Tidak ada titik tubuh yang valid pada {image_file}.\n"
            print(error_msg)
            log_file.write(error_msg)
            continue

        # Cari bounding box (xmin, ymin, xmax, ymax)
        x_coords, y_coords = zip(*valid_points)
        xmin, xmax = min(x_coords), max(x_coords)
        ymin, ymax = min(y_coords), max(y_coords)

        # Tambahkan margin ekstra (10% dari bounding box)
        margin_x = int((xmax - xmin) * 0.1)
        margin_y = int((ymax - ymin) * 0.1)

        xmin, ymin = max(0, xmin - margin_x), max(0, ymin - margin_y)
        xmax, ymax = min(xmax + margin_x, 9999), min(ymax + margin_y, 9999)  # 9999 untuk menjaga batas atas

        # Tentukan ukuran bounding box agar tetap 1:1
        box_size = max(xmax - xmin, ymax - ymin)

        # Hitung posisi baru agar tetap dalam batas gambar
        x_center, y_center = (xmin + xmax) // 2, (ymin + ymax) // 2
        xmin, ymin = x_center - box_size // 2, y_center - box_size // 2
        xmax, ymax = x_center + box_size // 2, y_center + box_size // 2

        # Pastikan bounding box tetap dalam ukuran gambar
        img = cv2.imread(input_image_path)
        h, w, _ = img.shape

        # Hitung padding yang diperlukan
        pad_top, pad_bottom, pad_left, pad_right = 0, 0, 0, 0

        if xmin < 0:
            pad_left = abs(xmin)
            xmin = 0
        if xmax > w:
            pad_right = xmax - w
            xmax = w
        if ymin < 0:
            pad_top = abs(ymin)
            ymin = 0
        if ymax > h:
            pad_bottom = ymax - h
            ymax = h

        # Crop gambar sesuai bounding box yang diperbaiki
        cropped_img = img[ymin:ymax, xmin:xmax]

        # Tambahkan padding abu-abu agar tetap 1:1 dan objek di tengah
        cropped_img = cv2.copyMakeBorder(
            cropped_img, pad_top, pad_bottom, pad_left, pad_right,
            cv2.BORDER_CONSTANT, value=(128, 128, 128)  # Warna abu-abu
        )

        # Resize gambar ke ukuran standar 256x256
        final_size = (256, 256)
        resized_img = cv2.resize(cropped_img, final_size)

        # Simpan hasil crop
        preprocessing_image_path = os.path.join(preprocessing_dir, f"{file_name}_preproc{file_ext}")

        # Ambil titik hidung dan bahu
        nose = get_coords(0)  # Hidung
        left_shoulder = get_coords(5)  # Bahu kiri
        left_elbow = get_coords(6)  # Siku kiri
        left_wrist = get_coords(7)  # Pergelangan tangan kiri
        left_knee = get_coords(13)  # Lutut kiri

        score = 0

        if nose and left_shoulder:
            if nose[0] < left_shoulder[0]:  
                score += 1  # Menghadap kiri
            else:
                score -= 1  # Menghadap kanan
        
        if left_shoulder and left_wrist:
            if left_shoulder[0] > left_wrist[0]:  
                score += 1  # Menghadap kiri
            else:
                score -= 1  # Menghadap kanan
        
        if left_shoulder and left_knee:
            if left_shoulder[0] > left_knee[0]:  
                score += 1  # Menghadap kiri
            else:
                score -= 1  # Menghadap kanan
        
        if left_shoulder and left_elbow:
            if left_shoulder[0] > left_elbow[0]:  
                score += 1  # Menghadap kiri
            else:
                score -= 1  # Menghadap kanan

        if score > 0:
            print("Menghadap kiri → Flip ke kanan")
            img = resized_img
            flipped_img = cv2.flip(img, 1)  # Flip horizontal
            cv2.imwrite(preprocessing_image_path, flipped_img)
        elif score < 0:
            cv2.imwrite(preprocessing_image_path, resized_img)
        else:
            error_msg = f"Skor netral, tidak bisa menentukan arah hadap pada {image_file}.\n"
            print(error_msg)
            log_file.write(error_msg)
            continue

# Ambil file gambar dari folder `preprocessing_images/`
image_files_output = [f for f in os.listdir(preprocessing_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
total_images_output = len(image_files_output)
failed_image_process = total_images - total_images_output
print(f"\nTotal file berhasil diproses = {total_images_output}")
print(f"Total file gagal diproses = {failed_image_process}")

print(f"\nSemua gambar berhasil diproses. Hasil disimpan di folder `preprocessing_images/`.\n")
print(f"Log error tersimpan di: {log_file_path}")

In [ ]:
import os
import shutil

# Path ke direktori output JSON dan preprocessing images
json_output_dir = os.path.abspath("../output_json")  # Folder JSON hasil OpenPose
preprocessing_dir = os.path.abspath("../preprocessing_images")  # Folder hasil crop

def clear_folder(folder_path):
    """Menghapus semua isi folder tanpa menghapus foldernya sendiri."""
    if os.path.exists(folder_path):
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)  # Hapus file atau symbolic link
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)  # Hapus folder beserta isinya
            except Exception as e:
                print(f"Gagal menghapus {file_path}: {e}")
    else:
        print(f"Folder {folder_path} tidak ditemukan!")

# Hapus isi folder output_json dan preprocessing_images
clear_folder(json_output_dir)
clear_folder(preprocessing_dir)

print("Semua isi folder 'output_json' dan 'preprocessing_images' telah dihapus.")